In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import patches, pyplot, pyplot as plt
# from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [10, 8]
import cv2
import xml.etree.ElementTree as ET 
import numpy as np
import sys
import warnings
warnings.filterwarnings("ignore")
from getBoundingBoxes import *
from corner_detector import *
from getBoxFeatures import *
from anms import *
from utils import *
from estimateAllTranslation import *
from estimateFeatureTranslation import *
from applyBoxTransform import *
from refreshFeatures import *
from anms_mod import*

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# cap = cv2.VideoCapture('vids/Easy.mp4')
# ret,firstFrame = cap.read()
# boxes = getBoundingBoxes()
# gray = cv2.cvtColor(firstFrame,cv2.COLOR_BGR2GRAY)

# boxesData = []

# for box in boxes:
#     boximg = gray[box[1]:box[3],box[0]:box[2]]
#     x,y = getBoxFeatures(boximg,box,100)
#     boxData = {
#         'coords': box,
#         '_x': x,
#         '_y': y
#     }
#     boxesData.append(boxData)

# currentFrame = firstFrame
# _,nextFrame = cap.read()

In [23]:
boxes = getBoundingBoxes('0210.xml')
currentFrame = cv2.imread('framesMed/0210.png')
nextFrame = cv2.imread('framesMed/0209.png')
gray = cv2.cvtColor(currentFrame,cv2.COLOR_BGR2GRAY)
boxesData = []
pts = 10
for box in boxes:
    boximg = gray[box[1]:box[3],box[0]:box[2]]
    x,y = getBoxFeatures(boximg,box,pts)
    boxData = {
        'coords': box,
        'x': x,
        'y': y,
        'startX': x.copy(),
        'startY': y.copy(),
        'startCoords': box,
        'valid': np.ones(x.shape[0],dtype='bool')
    }
    boxesData.append(boxData)

In [24]:
# Imag, Ix, Iy,Iori=findDerivatives(gray)
# plt.imshow(Imag)
# _x = boxesData[0]['_x']
# _y = boxesData[0]['_y']
for i in range(210,440,1):
    idx = str(i)
    idx = (idx.zfill(4))
    nextFrame = cv2.imread("framesMed/"+idx+".png")
    gray = cv2.cvtColor(currentFrame,cv2.COLOR_BGR2GRAY)

    for boxData in boxesData:
        print('----',np.sum(boxData['valid']))
        if np.sum(boxData['valid'])>=pts-5:
            boxData['x'],boxData['y'],boxData['valid'] = estimateAllTranslation(boxData['x'],boxData['y'],boxData['valid'],currentFrame,nextFrame)
        else:
            print('inside else')
            boxData['startCoords']=boxData['coords']
            box = boxData['coords'].astype(int)
            print(box)
            boximg = gray[box[1]:box[3],box[0]:box[2]]
            boxData['x'],boxData['y'],boxData['valid'] = refreshFeatures(boximg,box,boxData['x'], boxData['y'], boxData['valid'],pts)
            boxData['startX']=boxData['x']
            boxData['startY']=boxData['y'] 
            print('----after refresh',np.sum(boxData['valid']))
        startX = boxData['startX']
        startY = boxData['startY']
        plt.imshow(cv2.cvtColor(nextFrame,cv2.COLOR_BGR2RGB))
        fig = plt.gcf()
        ax1 = fig.add_subplot(111)
        ax1.scatter(boxData['x'][boxData['valid']],boxData['y'][boxData['valid']],c='r',s=1)
        # print(np.sum(boxData['valid']),end=" ")
        if np.sum(boxData['valid']) >= 4:
            boxprev = boxData['coords']
            boxData['coords'] = applyBoxTransform(startX,startY,boxData['x'],boxData['y'],boxData['coords'],boxData['valid'])
            coords = boxData['coords']
            # xmin,ymin,xmax,ymax
            if coords[0] > 0 and coords[1] > 0 and coords[2] < currentFrame.shape[1] and coords[3] < currentFrame.shape[0]:
                rect = patches.Rectangle((coords[0],coords[1]),coords[2]-coords[0],coords[3]-coords[1],linewidth=1,edgecolor='b',facecolor='none')
                ax1.add_patch(rect)
            else:
                print('homo fucked up')
                coords = boxprev
                boxData['coords']=boxprev
                rect = patches.Rectangle((coords[0],coords[1]),coords[2]-coords[0],coords[3]-coords[1],linewidth=1,edgecolor='b',facecolor='none')
                ax1.add_patch(rect)
    plt.savefig("outputsMedRev/"+idx+".png")
    fig.clf()
    print(idx)
    currentFrame = nextFrame
    
    
    
    
    
    
    
    
    
    
    
    
#     idx = str(i)
#     idx = (idx.zfill(4))
#     x,y=estimateAllTranslation(x,y,currentFrame,nextFrame)
#     currentFrame = nextFrame
# #     print(x.shape)
#     plt.imshow(cv2.cvtColor(nextFrame,cv2.COLOR_BGR2RGB))
#     fig = plt.gcf()
#     ax1 = fig.add_subplot(111)
#     ax1.scatter(x,y,c='r',s=0.3)
#     plt.savefig("outputs/"+idx+".png")
#     nextFrame = cv2.imread("framesMed/"+idx+".png")
# #     fig.clf()
#     print(idx)
# print(X-x)
# print(Y-y)

---- 10
0210
---- 10
0211
---- 10
0212
---- 9
0213
---- 8
homo fucked up
0214
---- 7
0215
---- 7
homo fucked up
0216
---- 6
0217
---- 5
0218
---- 5
0219
---- 5
0220
---- 4
inside else
[ 95 461 457 506]
(10,)
----after refresh 10
0221
---- 10
0222
---- 9
0223
---- 9
0224
---- 7
0225
---- 6
0226
---- 5
0227
---- 3
inside else
[ 81 438 431 516]
(10,)
----after refresh 10
0228
---- 10
0229
---- 10
0230
---- 10
0231
---- 10
0232
---- 8
0233
---- 8
0234
---- 7
homo fucked up
0235
---- 7
homo fucked up
0236
---- 7
homo fucked up
0237
---- 7
homo fucked up
0238
---- 7
homo fucked up
0239
---- 7
homo fucked up
0240
---- 7
homo fucked up
0241
---- 7
0242
---- 6
homo fucked up
0243
---- 5
homo fucked up
0244
---- 4
inside else
[ 13 388 348 460]
(10,)
----after refresh 10
0245
---- 10
0246
---- 9
0247
---- 9
homo fucked up
0248
---- 8
homo fucked up
0249
---- 8
homo fucked up
0250
---- 7
homo fucked up
0251
---- 7
homo fucked up
0252
---- 7
homo fucked up
0253
---- 6
homo fucked up
0254
---- 6
hom

<Figure size 720x576 with 0 Axes>

In [ ]:
plt.imshow(gray)
fig = plt.gcf()
ax1 = fig.add_subplot(111)
ax1.scatter(x,y,c='r',s=1)

In [ ]:
plt.imshow(cv2.cvtColor(nextFrame,cv2.COLOR_BGR2GRAY))
fig = plt.gcf()
ax1 = fig.add_subplot(111)
ax1.scatter(X,Y,c='r',s=1)


In [ ]:
print(X-x)
print(Y-y)

In [ ]:

h = est_homography(x,y,X,Y)

In [ ]:
pt = np.array([x[0],y[0],1]).T
newpt = h@pt
newpt = newpt/newpt[2]
print(newpt)
print(np.array([X[0],Y[0],1]))

In [ ]:
n = x.shape[0]
stk = np.vstack((x,y,np.ones(n)))

newstk = np.vstack((X,Y,np.ones(n)))

In [ ]:
res=h@stk
res=res/res[2,:]

In [ ]:
# bdata=boxesData[0]['coords']
X,Y,_box = applyBoxTransform(x,y,X,Y,box)

In [ ]:
_box

In [ ]:
box

In [ ]:
h